In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt

from argparse import Namespace

from training import ModelTrainer, TrajectoryGenerator

from models import SingleTimeStep

from experiments import entropy_loss, entropy_infer, force_loss, force_infer

from free_diffusion import params, simulate_free_diffusion_underdamped, realepr, traj_dEnt

### EXAMPLE TRAIN/INFERENCE IF LOSS IS FOR FORCE

In [ ]:
training_options = Namespace()
model_options = Namespace()

training_options.n_epoch = 200
training_options.epoch_s = 10_000

training_options.n_iter = 1
training_options.iter_s = 10_000

training_options.n_infer = 1
training_options.infer_s = 10_000

training_options.lr = 1E-4
training_options.wd = 1E-5

model_options.n_input = 2
model_options.n_hidden = 512
model_options.n_output = 1
# this is how many hidden layers to use
model_options.num_inner =2

In [ ]:
#params stores information for the sim
print(params)

In [ ]:
#gamma is the force since we are in free diffusion
params['gamma'] = 1

#this is the initial distribution params for the free diffusion
params['init'] = [1,1,1]

# we want to test coarse data, lets say we only see every 10 steps
params['coarse'] = 10
params['dt'] = .001

#how long of a trajectory to simulate 
# we'll get the force for 1 step only, obviously
# it is set to 20 because we need to simulater 20 steps to get one step at coarse = 10 with inferring velocity
params['num_steps'] = 20
print(params)

In [ ]:
WeightFunction = SingleTimeStep(model_options)
FreeDiffusion = TrajectoryGenerator(simulate_free_diffusion_underdamped, params)

#need to set this, so that the simulator knows to return estimated velocity insead of the real one
FreeDiffusion.infer_velocity = True

optimizer = torch.optim.Adam
#optimizer = torch.optim.SGD

Force = ModelTrainer(WeightFunction, FreeDiffusion, optimizer, force_loss, force_infer, training_options)

In [ ]:
untrained_output, untrained_test_trajectories = Force.infer(return_trajectories=True)

In [ ]:
# can edit training options here if you want
#training_options.n_epoch = 150
#training_options.epoch_s = 10_000
#training_options.n_iter = 1
#training_options.iter_s = 10_000

Force.train()

In [ ]:
fig, ax = plt.subplots(1,2)
plt.close()
Force.plot_training_loss(ax=ax)

In [ ]:
output, test_trajectories = Force.infer(return_trajectories = True)

In [ ]:
fig, ax = plt.subplots(1,2, sharex=True, sharey=True)
vmin, vmax = test_trajectories[0].min(), test_trajectories[0].max()

ut_traj = untrained_test_trajectories[0][...,1].ravel()
ut_out = untrained_output[0].ravel()

tr_traj = test_trajectories[0][...,1].ravel()
tr_out = output[0].ravel()

ax[0].scatter(ut_traj, ut_out, s=1, alpha=.8, c='r', label='untrained model')
ax[1].scatter(tr_traj,tr_out, s=1, alpha=.8,c='b', label='trained model')

for i in range(2):
    ax[i].plot(np.linspace(vmin,vmax,10),-params['gamma']*np.linspace(vmin,vmax,10), c='k', label='-v*$\\gamma$')
    ax[i].legend()
ax[0].set_xlabel('v')
ax[0].set_xlabel('F')

### EXAMPLE TRAIN/INFERENCE IF LOSS IS FOR EP

In [ ]:

training_options = Namespace()
model_options = Namespace()

training_options.n_epoch = 10
training_options.epoch_s = 50_000

training_options.n_iter = 20
training_options.iter_s = 25_000

training_options.n_infer = 20
training_options.infer_s = 50_000

training_options.lr = 1E-4
training_options.wd = 1E-5

model_options.n_input = 2
model_options.n_hidden = 512
model_options.n_output = 1
model_options.num_inner = 2

In [ ]:
# how long of a trajectory to get the EP for, here we can accomodate more than one step
# but it is still rate based ultimately
params['num_steps'] = 1

#this is the initial distribution params for the free diffusion
params['init'] = [.6,.8,.6]


#for non coarse data
params['coarse'] = 1
params['dt'] = .001

print(params)

In [ ]:
WeightFunction = SingleTimeStep(model_options)
FreeDiffusion = TrajectoryGenerator(simulate_free_diffusion_underdamped, params)

#by default, velocity is not inferred so no need to change
print('inferring velocity:',FreeDiffusion.infer_velocity)

optimizer = torch.optim.Adam
#optimizer = torch.optim.SGD

EntProd = ModelTrainer(WeightFunction, FreeDiffusion, optimizer, entropy_loss, entropy_infer, training_options)

In [ ]:
EntProd.train()

In [ ]:
fig, ax = plt.subplots(1,2)
plt.close()
EntProd.plot_training_loss(ax=ax)

In [ ]:
output = EntProd.infer(return_trajectories=False)

In [ ]:
# calculating real average EP 
resolution = 1_000
T = params['Dt']*int(params['num_steps']/params['coarse'])
ents = realepr( np.linspace(0, T, resolution),*params['init'] )*(T/resolution)
ent_production = sum(ents)
print(ent_production)

In [ ]:
fig, ax = plt.subplots()
error = np.array(output)/ent_production - 1
m, s = np.mean(error), np.std(error)
s /= np.sqrt(len(error))
plt.plot(error, linestyle='none', marker='D')
for l in [m, m-3*s, m+3*s]:
    ax.axhline(l, c='k', linewidth=.75)

ax.set_xlabel('trial')
ax.set_ylabel('relative error')

ax.axhline(0, c='k', linestyle='--', linewidth=2)
